# Table of Contents
 <p><div class="lev1 toc-item"><a href="#Modify-ecoinvent" data-toc-modified-id="Modify-ecoinvent-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Modify ecoinvent</a></div><div class="lev1 toc-item"><a href="#Prepare-additional-datasets" data-toc-modified-id="Prepare-additional-datasets-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Prepare additional datasets</a></div><div class="lev2 toc-item"><a href="#import-from-excel" data-toc-modified-id="import-from-excel-21"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>import from excel</a></div><div class="lev1 toc-item"><a href="#Import-new-datasets-and-ecoinvent-into-Wurst-format" data-toc-modified-id="Import-new-datasets-and-ecoinvent-into-Wurst-format-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Import new datasets and ecoinvent into Wurst format</a></div><div class="lev1 toc-item"><a href="#Define-Image-scenarios-and-years-to-be-used:" data-toc-modified-id="Define-Image-scenarios-and-years-to-be-used:-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Define Image scenarios and years to be used:</a></div><div class="lev1 toc-item"><a href="#Modify-all-electricity-generation-and-market-datasets:" data-toc-modified-id="Modify-all-electricity-generation-and-market-datasets:-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Modify all electricity generation and market datasets:</a></div><div class="lev1 toc-item"><a href="#Run-standard-ecoinvent-through-the-same-code-without-the-changes-to-make-sure-all-databases-are-in-same-format" data-toc-modified-id="Run-standard-ecoinvent-through-the-same-code-without-the-changes-to-make-sure-all-databases-are-in-same-format-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Run standard ecoinvent through the same code without the changes to make sure all databases are in same format</a></div>

# Modify ecoinvent

In this notebook we create new versions of ecoinvent to represent the future!

We create 2 new versions of ecoinvent.

1. ecoinvent 2040 BAU (Baseline), which is a 2040 version of ecoinvent based on the IMAGE BAU scenario
2. ecoinvent 2040 450 (ClimPol), which is a 2040 version of ecoinvent based on the IMAGE 450 ppm scenario


In [1]:
from IPython.core.display import HTML
HTML("<style>.container { width:100% !important; }</style>")

In [2]:
%matplotlib inline

Import useful libraries and set project:

In [3]:
%run initialize_notebook.ipynb

Import functions that are written on top of Wurst:

In [4]:
%run _Functions_to_modify_ecoinvent.ipynb

In [5]:
databases

Databases dictionary with 10 object(s):
	CSP
	Carma CCS
	Cars
	Cars ecoinvent
	Energy Storage
	biosphere3
	ecoinvent
	ecoinvent_2040_450
	ecoinvent_2040_BAU
	ecoinvent_unchanged

# Prepare additional datasets

We import some additional datasets for processes such as CSP and carbon capture and storage electricity. These have been prepared in Simapro and exported. We have cleaned up the datasets and stored them in excel using Brightway input output functionality. We read them from excel here:

## import from excel

In [6]:
for k, fp in {'Carma CCS': 'lci-Carma-CCS.xlsx', 'CSP': 'lci-CSP.xlsx'}.items():
   
    if k not in databases:
        
        sp = ExcelImporter(fp)
        sp.apply_strategies()
        sp.match_database(fields=["name", "unit", "location"]) 
        sp.match_database('ecoinvent_unchanged',  fields=["reference product", "name", "unit", "location"]) 
        sp.match_database('ecoinvent_unchanged',  fields=[ "name", "unit", "location"])
        sp.write_database()
        del sp
        
        lca = LCA({Database(k).random(): 1}, method=lcia_methods['CC'])
        lca.lci()
        lca.lcia()
        print(lca.score)

In [7]:
databases

Databases dictionary with 10 object(s):
	CSP
	Carma CCS
	Cars
	Cars ecoinvent
	Energy Storage
	biosphere3
	ecoinvent
	ecoinvent_2040_450
	ecoinvent_2040_BAU
	ecoinvent_unchanged

# Import new datasets and ecoinvent into Wurst format

We are using the constructive geometries library (by Chris Mutel), which is still in it's infancy. There are some naming inconsistencies still, which we correct for in the following cell:

In [8]:
#these locations aren't found correctly by the constructive geometries library - we correct them here:
fix_names= {'CSG' : 'CN-CSG',
            'SGCC': 'CN-SGCC',
             
             'RFC' : 'US-RFC',
             'SERC' : 'US-SERC',
             'TRE': 'US-TRE',
             'ASCC': 'US-ASCC',
             'HICC': 'US-HICC',
             'FRCC': 'US-FRCC',
             'SPP' : 'US-SPP',
            'MRO, US only' : 'US-MRO', 
             'NPCC, US only': 'US-NPCC', 
             'WECC, US only': 'US-WECC',
             
             'IAI Area, Africa':'IAI Area 1, Africa',
             'IAI Area, South America':'IAI Area 3, South America', 
             'IAI Area, Asia, without China and GCC':'IAI Area 4&5, without China', 
             'IAI Area, North America, without Quebec':'IAI Area 2, without Quebec',
             'IAI Area, Gulf Cooperation Council':'IAI Area 8, Gulf'
            }

fix_names_back = {v:k for k,v in fix_names.items()}


def rename_locations(db, name_dict):
    for ds in db:
        if ds['location'] in name_dict:
            ds['location'] = name_dict[ds['location']]

        for exc in technosphere(ds):
            if exc['location'] in name_dict:
                exc['location'] = name_dict[exc['location']]
    


# Define Image scenarios and years to be used:

Here we set the scenarios and years for which we want to use when creating a new version of ecoinvent:

In [9]:
database_dict = collections.defaultdict(dict)

database_dict['ecoinvent_2040_BAU'] = {'year' : 2040 , 'scenario' : 'BAU'}
database_dict['ecoinvent_2040_450'] = {'year' : 2040 , 'scenario' : '450'}    

# Modify all electricity generation and market datasets:

This cell uses the Wurst functionality as well as the functions that we wrote in the notebook "_Functions_to_modify_ecoinvent.ipynb" to make the changes to ecoinvent:

In [11]:
for key in pyprind.prog_bar(database_dict.keys()):
    
    
    db = extract_brightway2_databases(['CSP','Carma CCS','ecoinvent_unchanged'])
    default_global_location(db)
    fix_unset_technosphere_and_production_exchange_locations(db)
    #set_global_location_for_additional_datasets(db)
    remove_nones(db)
    rename_locations(db, fix_names)
    #add_new_locations_to_added_datasets(input_db)
    #regionalize_added_datasets(input_db)
    add_negative_CO2_flows_for_biomass_CCS(db)
    
    year = database_dict[key]['year']
    scenario = database_dict[key]['scenario']
    
    print(key)
    # Electricity markets:
    print('Changing electricity Markets')
    market_changes = update_electricity_markets(db, year, scenario)
    
    #Electricity generation datasets:
    print('Changing ecoinvent electricity generation datasets')
    technology_changes = update_electricity_datasets_with_image_data(db, year, scenario, agg_func=np.average, update_efficiency = True, update_emissions = True)
    
    #Electricity generation datasets from project Carma
    print('Changing Carma electricity datasets')
    modify_all_carma_electricity_datasets(db, year, scenario)
    
    print('Saving changes to excel')
    tech_df = pd.DataFrame.from_dict(technology_changes)
    tech_df.index = pd.MultiIndex.from_tuples(tech_df.index)
    tech_df=tech_df.T
    tech_df = tech_df.set_index([('meta data','Image technology'),('meta data','name'),('meta data','location')], drop=True).sort_index()
    market_df = pd.DataFrame.from_dict(market_changes).T
    market_df = market_df.set_index([('meta data','name'),('meta data','location')], drop=True).sort_index().T

    writer = pd.ExcelWriter('electricity changes '+ str(year) + ' '+ scenario + '.xlsx')
    market_df.to_excel(writer, sheet_name = 'markets')
    for tech in tech_df.index.levels[0]:
        tech_df.loc[tech].dropna(how='all', axis=1).swaplevel(i=0,j=1, axis=1).T.to_excel(writer, sheet_name = tech)
    writer.save()
    del tech_df
    del market_df
    del writer
    
    
    rename_locations(db, fix_names_back)
    if key in databases:
        del databases[key]

    write_brightway2_database(db, key)
    del db
    
        
    lca = LCA({Database(key).random(): 1}, method=lcia_methods['CC'])
    lca.lci()
    lca.lcia()
    print(lca.score)


Getting activity data


100%|██████████████████████████████████████████████████████████████████████████████████████████████| 15060/15060 [00:02<00:00, 6565.86it/s]


Adding exchange data to activities


100%|███████████████████████████████████████████████████████████████████████████████████████████| 523406/523406 [00:33<00:00, 15576.54it/s]


Filling out exchange data


...

100%|██████████████████████████████████████████████████████████████████████████████████████████████| 15060/15060 [00:04<00:00, 3684.49it/s]


ecoinvent_2040_450
Changing electricity Markets
Changing ecoinvent electricity generation datasets
Changing  Coal ST
Changing  Coal CHP
Changing  Natural gas OC
Changing  Natural gas CC
Changing  Natural gas CHP
Changing  Oil ST
Changing  Oil CC
Changing  Oil CHP
Changing  Biomass ST
Changing  Biomass CHP
Changing  Biomass CC
Changing  Nuclear
Changing Carma electricity datasets
Saving changes to excel
15060 datasets
606462 exchanges
0 unlinked exchanges
  


Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:01:45


Title: Writing activities to SQLite3 database:
  Started: 12/18/2018 09:26:39
  Finished: 12/18/2018 09:28:24
  Total time elapsed: 00:01:45
  CPU %: 57.10
  Memory %: 46.87
Created database: ecoinvent_2040_450
0.6962862999117959


0% [##] 100% | ETA: 00:00:00
Total time elapsed: 00:42:59


# Run standard ecoinvent through the same code without the changes to make sure all databases are in same format

In [9]:
key = 'ecoinvent'
  
db = extract_brightway2_databases(['CSP','Carma CCS','ecoinvent_unchanged'])
default_global_location(db)
fix_unset_technosphere_and_production_exchange_locations(db)
#set_global_location_for_additional_datasets(db)
remove_nones(db)
rename_locations(db, fix_names)
#add_new_locations_to_added_datasets(input_db)
#regionalize_added_datasets(input_db)
add_negative_CO2_flows_for_biomass_CCS(db)
    
rename_locations(db, fix_names_back)
if key in databases:
    del databases[key]

write_brightway2_database(db, key)
del db
    
        
lca = LCA({Database(key).random(): 1}, method=lcia_methods['CC'])
lca.lci()
lca.lcia()
print(lca.score)


Getting activity data


100%|██████████████████████████████████████████████████████████████████████████████████████████████| 15060/15060 [00:02<00:00, 5771.75it/s]


Adding exchange data to activities


100%|███████████████████████████████████████████████████████████████████████████████████████████| 523406/523406 [00:36<00:00, 14512.21it/s]


Filling out exchange data


100%|██████████████████████████████████████████████████████████████████████████████████████████████| 15060/15060 [00:03<00:00, 3871.08it/s]


15060 datasets
523413 exchanges
0 unlinked exchanges
  


Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:01:46


Title: Writing activities to SQLite3 database:
  Started: 12/21/2018 10:43:30
  Finished: 12/21/2018 10:45:17
  Total time elapsed: 00:01:46
  CPU %: 46.50
  Memory %: 33.81
Created database: ecoinvent
230638447.19546476


In [10]:
databases

Databases dictionary with 10 object(s):
	CSP
	Carma CCS
	Cars
	Cars ecoinvent
	Energy Storage
	biosphere3
	ecoinvent
	ecoinvent_2040_450
	ecoinvent_2040_BAU
	ecoinvent_unchanged